<a href="https://colab.research.google.com/github/shuchimishra/Model_deployment_using_Kubernetes_and_Docker/blob/main/Running_Flask_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Copy the model output and tokens from Google drive to current location

In [2]:
!cp -r "/content/drive/MyDrive/Data Science & Machine Learning/MLOps/Repository/model_output" "/content"

In [3]:
!ls "/content/model_output"

sa_encoder.vocab.tokens  sentiment_analysis.hdf5


In [4]:
# !ls '/content/drive/My Drive/Colab Notebooks/models/'

# Import all libraries

In [5]:
import socket


## View Socket details

In [6]:
print(socket.gethostname())
print(socket.gethostbyname(socket.gethostname()))

173deae104a1
172.28.0.12


In [9]:
del app, model, text_encoder, padding_size, request

NameError: name 'app' is not defined

In [12]:
from flask import Flask, jsonify, make_response, request

In [13]:
#Initialize Flask app
app = Flask(__name__)

In [14]:
import tensorflow_datasets as tfds
import tensorflow as tf

import threading
import json
import requests

In [15]:
padding_size = 1000


# Load the model and tokens
model = tf.keras.models.load_model('/content/model_output/sentiment_analysis.hdf5')
text_encoder = tfds.deprecated.text.TokenTextEncoder.load_from_file("/content/model_output/sa_encoder.vocab") #this feature is deprecated; for demo purpose only

In [16]:
@app.route('/')
def hello():
  return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})


threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':8000}).start()

 * Serving Flask app '__main__'
 * Debug mode: off


Model and Vocabalory loaded.......
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   WARNING: This is a development server. Do not use it in a production deployment.
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:6000/ (Press CTRL+C to quit)

In [17]:

req = requests.get("http://172.28.0.12:8000") #http://172.28.0.2:6000/
print(req.status_code)
print(req.text)

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8000
 * Running on http://172.28.0.12:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:172.28.0.12 - - [18/Jun/2024 00:11:17] "GET / HTTP/1.1" 200 -


200
I am alive!


In [18]:
seclassifier = "http://172.28.0.12:8000/seclassifier"

In [19]:
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"Still working my way through it but definitely changes your view on investment. Wish it was available on Audible"}
req = requests.post(seclassifier,  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)

Still working my way through it but definitely changes your view on investment. Wish it was available on Audible
1/1 [==============================] - 4s 4s/step


INFO:werkzeug:172.28.0.12 - - [18/Jun/2024 00:11:20] "POST /seclassifier HTTP/1.1" 200 -


200
{"predictions ":[[0.8370057344436646]],"sentiment ":"positive"}



In [20]:
app.app_context().push()
headers = {'Content-type': 'application/json', 'Accept': 'text/plain'}
data = {"text":"I thought this will be good one. But I was wrong"}
req = requests.post(seclassifier,  data=json.dumps(data), headers=headers)

print(req.status_code)
print(req.text)

I thought this will be good one. But I was wrong
1/1 [==============================] - 1s 634ms/step


INFO:werkzeug:172.28.0.12 - - [18/Jun/2024 00:11:21] "POST /seclassifier HTTP/1.1" 200 -


200
{"predictions ":[[-0.8676939606666565]],"sentiment ":"Negative"}



In [21]:
!ps -eaf | grep python

root          61       6  0 Jun17 ?        00:00:07 [python3] <defunct>
root          62       6  0 Jun17 ?        00:00:00 python3 /usr/local/bin/colab-fileshim.py
root          82       6  0 Jun17 ?        00:00:06 /usr/bin/python3 /usr/local/bin/jupyter-notebook
root         596     592  0 Jun17 ?        00:00:00 python3 /opt/google/drive/drive-filter.py
root        9142      82 21 00:10 ?        00:00:15 /usr/bin/python3 -m colab_kernel_launcher -f /ro
root        9163       1  0 00:10 ?        00:00:00 /usr/bin/python3 /usr/local/lib/python3.10/dist-
root        9526    9142  0 00:11 ?        00:00:00 /bin/bash -c ps -eaf | grep python
root        9528    9526  0 00:11 ?        00:00:00 grep python


In [22]:
import flask
flask.__version__

'2.2.5'

In [23]:
!pip install flask-ngrok

In [31]:
!pip install pyngrok==4.1.1

  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15964 sha256=7fbcf24859c963d5406a76a35fd755bd27f5558bafa052c16667d5b5dc254f9c
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok


In [ ]:
ngrok_token = "" # @param {type:"string"}
import tensorflow_datasets as tfds
import tensorflow as tf
from flask import Flask, jsonify, make_response, request
from flask_ngrok import run_with_ngrok
!ngrok config add-authtoken ngrok_token #Important code

app = Flask(__name__)
run_with_ngrok(app)

padding_size = 1000
model = tf.keras.models.load_model('/content/model_output/sentiment_analysis.hdf5')
text_encoder = tfds.deprecated.text.TokenTextEncoder.load_from_file("/content/model_output/sa_encoder.vocab")

print('Model and Vocabalory loaded.......')

@app.route("/")
def hello():
    return "I am alive!"

def pad_to_size(vec, size):
    zeros = [0] * (size - len(vec))
    vec.extend(zeros)
    return vec


def predict_fn(predict_text, pad_size):
    encoded_text = text_encoder.encode(predict_text)
    encoded_text = pad_to_size(encoded_text, pad_size)
    encoded_text = tf.cast(encoded_text, tf.int64)
    predictions = model.predict(tf.expand_dims(encoded_text, 0))

    return (predictions.tolist())


@app.route('/seclassifier', methods=['POST'])
def predict_sentiment():
    text = request.get_json()['text']
    print(text)
    predictions = predict_fn(text, padding_size)
    sentiment = 'positive' if float(''.join(map(str,predictions[0]))) > 0 else 'Negative'
    return jsonify({'predictions ':predictions, 'sentiment ': sentiment})

if __name__ == '__main__':
    app.run()

NAME:
   ngrok - tunnel local ports to public URLs and inspect traffic

DESCRIPTION:
    ngrok exposes local networked services behinds NATs and firewalls to the
    public internet over a secure tunnel. Share local websites, build/test
    webhook consumers and self-host personal services.
    Detailed help for each command is available with 'ngrok help <command>'.
    Open http://localhost:4040 for ngrok's web interface to inspect traffic.

EXAMPLES:
    ngrok http 80                    # secure public URL for port 80 web server
    ngrok http -subdomain=baz 8080   # port 8080 available at baz.ngrok.io
    ngrok http foo.dev:80            # tunnel to host:port instead of localhost
    ngrok http https://localhost     # expose a local https server
    ngrok tcp 22                     # tunnel arbitrary TCP traffic to port 22
    ngrok tls -hostname=foo.com 443  # TLS traffic for foo.com to port 443
    ngrok start foo bar baz          # start tunnels from the configuration file

VERSI

INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-18:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

Copy the model output and tokens from Google drive to current location

In [ ]:
!cp -r "/content/drive/MyDrive/Data Science & Machine Learning/MLOps/Repository/model_output" "/content"

In [ ]:
!ls "/content/model_output"

sa_encoder.vocab.tokens  sentiment_analysis.hdf5
